# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) 160306, Piotr Franc <br>
2) 160288, Paweł Charkiewicz

In [2]:
from pulp import *  
import numpy as np
import pandas as pd
import itertools
from IPython.display import display

# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [3]:
problem = LpProblem("Problem", LpMaximize)

x1 = LpVariable("x1", lowBound=0)
x2 = LpVariable("x2", lowBound=0)
x3 = LpVariable("x3", lowBound=0)


obj = 4*x1 + 2*x2 + 3*x3
problem += obj

problem.addConstraint(LpConstraint(x1+x2<=10, name="c1"))
problem.addConstraint(LpConstraint(2*x2+x3<=12, name="c2"))
problem.addConstraint(LpConstraint(3*x1+2*x3<=15, name="c3"))
problem.addConstraint(LpConstraint(x1+x2+x3<=20, name="c4"))

problem.solve()
print("Status:", LpStatus[problem.status])
print("Objective Value:", value(problem.objective))
print("Variable Values:")
for v in problem.variables():
    print(f"  {v.name}: {v.varValue}")

x1 = x1.varValue
x2 = x2.varValue
x3 = x3.varValue
print(x1+x2,10)
print(2*x2+x3,12)
print(3*x1+2*x3,15)
print(x1+x2+x3,20)

# For some reason the problem is not feasible, however the checks above show that all constraints are satisfied

Status: Infeasible
Objective Value: 31.428571379999998
Variable Values:
  x1: 4.4285714
  x2: 5.5714286
  x3: 0.85714286
10.0 10
12.00000006 12
14.99999992 15
10.85714286 20


1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

In [13]:
# Initialize the data here:

# x1 x2 x3 | x4 x5 x6 x7

cx = [4, 2, 3, 0, 0, 0, 0]
c = cx[:3]
b = [10,12,15,20]
A = [
    [1, 1, 0],
    [0, 2, 1],
    [3, 0, 2],
    [1, 1, 1]
]
M = np.array([
    [1, 1, 0, 1, 0, 0, 0],
    [0, 2, 1, 0, 1, 0, 0],
    [3, 0, 2, 0, 0, 1, 0],
    [1, 1, 1, 0, 0, 0, 1]
])

count_f = 0
count_if = 0

for x in range(7):
    for y in range(x+1,7):
        for z in range(y+1,7):
            for w in range(z+1,7):
                cb = [cx[x], cx[y], cx[z], cx[w]]
                B = M[:,[x,y,z,w]]
                
                if abs(np.linalg.det(B)) < EPS:
                    print("Basis is singular -> skipping")
                    continue
                
                B_inv = np.linalg.inv(B)
                if(all(np.round(cb @ B_inv @ A - c, 3) >= 0) and all(np.round(B_inv @ b,3) >= 0)):
                    print("============================")
                    print(f"Checking basis: {x}, {y}, {z}, {w}")
                    print('Values for base variables: ')
                    print(B_inv @ b)
                    print("Values for slack variables: ")
                    print(np.round(cb @ B_inv))
                    print("Z: ")
                    print(cb @ B_inv @ b)
                    print("Optimality test: ", all(np.round(cb @ B_inv @ A - c, 3) >= 0))
                    print(np.round(cb @ B_inv @ A - c,3))
                    print("Feasibility test: ", all(np.round(B_inv @ b,3) >= 0))
                    print(np.round(B_inv @ b,3))
                    print("============================")
                if all(np.round(B_inv @ b,3) >= 0):
                    count_f += 1
                else:
                    count_if += 1

print("How many there should be - combination of 7 choose 4: ", 7*6*5*4//(4*3*2*1))
print("Basis is singular count: ", 35 - (count_f + count_if))
print("Number of feasible solutions: ", count_f)
print("Number of infeasible solutions: ", count_if)


Checking basis: 0, 1, 2, 6
Values for base variables: 
[4.42857143 5.57142857 0.85714286 9.14285714]
Values for slack variables: 
[1. 1. 1. 0.]
Z: 
31.428571428571427
Optimality test:  True
[-0.  0.  0.]
Feasibility test:  True
[4.429 5.571 0.857 9.143]
Basis is singular -> skipping
Basis is singular -> skipping
Basis is singular -> skipping
Basis is singular -> skipping
How many there should be - combination of 7 choose 4:  35
Basis is singular count:  4
Number of feasible solutions:  8
Number of infeasible solutions:  23


<b> Important note: the below is just a proposition. You can solve the problem in your own way. </b>

You can define an auxiliary method constructing a final simplex tableau for a given base.  Here, "base" is a list of columns (integers) for the base. Note that the functions in python can return multiple objects and you can use this functionality to return<br>
- the inversed base<br>
- coefficients in the row 0 for slack variables<br>
- right side values (except the objective function value)<br>
- the objective function value<br>
- the coefficients for decision variables in row 0 <br>
- the coefficients for decision variables in rows 1+<br>

Note that if BI cannot be built (it is possible), the method may return None in order to notify the executive method about this exception. 



In [5]:
# Did it all above...

# def getFinalTableau(base, c, b, A, M):
#     # TODO
#     # return None


In [6]:
### Solve the problem here
### TODO

## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal?

In [7]:
problem_dual = LpProblem("Problem", LpMinimize)

y1 = LpVariable("y1", lowBound=0)
y2 = LpVariable("y2", lowBound=0)
y3 = LpVariable("y3", lowBound=0)
y4 = LpVariable("y4", lowBound=0)


obj = 10*y1 + 12*y2 + 15*y3 + 20*y4
problem_dual += obj

problem_dual.addConstraint(LpConstraint(y1+3*y3+y4>=4, name="c1"))
problem_dual.addConstraint(LpConstraint(y1+2*y2+y4>=2, name="c2"))
problem_dual.addConstraint(LpConstraint(y2+2*y3+y4>=3, name="c3"))

In [8]:
problem_dual.solve()
print("Status:", LpStatus[problem_dual.status])
print("Objective Value:", value(problem_dual.objective))
print("Variable Values:")
for v in problem_dual.variables():
    print(f"  {v.name}: {v.varValue}")

y1 = y1.varValue
y2 = y2.varValue
y3 = y3.varValue
y4 = y4.varValue
print(y1+3*y3+y4,4)
print(y1+2*y2+y4,2)
print(y2+2*y3+y4,3)

Status: Optimal
Objective Value: 31.42857072
Variable Values:
  y1: 0.57142857
  y2: 0.71428571
  y3: 1.1428571
  y4: 0.0
3.9999998700000003 4
1.99999999 2
2.99999991 3


2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions wlong with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [9]:
import numpy as np
import pandas as pd

# Initialize the data here:
PRIMAL_DUAL_SOLUTIONS = []
cx = [4, 2, 3, 0, 0, 0, 0]
c = cx[:3]
b = [10, 12, 15, 20]
A = [
    [1, 1, 0],
    [0, 2, 1],
    [3, 0, 2],
    [1, 1, 1]
]
M = np.array([
    [1, 1, 0, 1, 0, 0, 0],
    [0, 2, 1, 0, 1, 0, 0],
    [3, 0, 2, 0, 0, 1, 0],
    [1, 1, 1, 0, 0, 0, 1]
])

def optimality(f, o):
    if f == True and o == True:
        return "Optimal"
    elif f == True and o == False:
        return "Suboptimal"
    elif f == False and o == True:
        return "Superoptimal"
    else:
        return "-"

for x in range(7):
    for y in range(x+1, 7):
        for z in range(y+1, 7):
            for w in range(z+1, 7):
                cb = [cx[x], cx[y], cx[z], cx[w]]
                B = M[:, [x, y, z, w]]

                if np.linalg.det(B) == 0:
                    continue
                    
                B_inv = np.linalg.inv(B)
                Z = cb @ B_inv @ b
                base = B_inv @ b
                
                y_values = cb @ B_inv
                surplus_values = cb @ B_inv @ A - c
                
                row = [0] * 21
                row[0] = Z
                
                row[x+1] = base[0]
                row[y+1] = base[1]
                row[z+1] = base[2]
                row[w+1] = base[3]
                
                row[14] = y_values[0]
                row[15] = y_values[1]
                row[16] = y_values[2]
                row[17] = y_values[3]
                row[18] = surplus_values[0]
                row[19] = surplus_values[1]
                row[20] = surplus_values[2]
                
                row[8] = all(np.round(row[1:8], 3) >= 0)
                row[9] = all(np.round(surplus_values, 3) >= 0)
                
                row[12] = all(np.round(row[14:21], 3) >= 0)
                row[13] = all(np.round(row[4:8], 3) >= 0) 
            
                
                row[10] = optimality(row[8], row[9])
                row[11] = optimality(row[12], row[13])
                
                PRIMAL_DUAL_SOLUTIONS.append(row)
                
PRIMAL_DUAL_SOLUTIONS = sorted(PRIMAL_DUAL_SOLUTIONS, key=lambda x: x[0], reverse=True)

## DISPLAY STORED PAIRS OF SOLUTIONS
df = pd.DataFrame(PRIMAL_DUAL_SOLUTIONS, columns = ["Z", "x1", "x2", "x3", "slack1", "slack2", "slack3", "slack4", "P_F", "P_O", "P_STATE", "D_STATE", "D_F", "F_O", "y1", "y2", "y3", "y4", "sur1", "sur2", "sur3"])

display(df.style.set_properties(**{
       'width': '230px',
       'max-width': '230px',
   }))

,Z,x1,x2,x3,slack1,slack2,slack3,slack4,P_F,P_O,P_STATE,D_STATE,D_F,F_O,y1,y2,y3,y4,sur1,sur2,sur3
0,80.000000,20.000000,0.000000,0.000000,-10.000000,12.000000,-45.000000,0.000000,False,True,Superoptimal,Suboptimal,True,False,0.000000,0.000000,0.000000,4.000000,0.000000,2.000000,1.000000
1,76.000000,10.000000,0.000000,12.000000,0.000000,0.000000,-39.000000,-2.000000,False,True,Superoptimal,Suboptimal,True,False,4.000000,3.000000,0.000000,0.000000,0.000000,8.000000,0.000000
2,70.000000,10.000000,0.000000,10.000000,0.000000,2.000000,-35.000000,0.000000,False,True,Superoptimal,Suboptimal,True,False,1.000000,-0.000000,0.000000,3.000000,0.000000,2.000000,0.000000
3,68.000000,41.000000,33.000000,-54.000000,-64.000000,0.000000,0.000000,0.000000,False,True,Superoptimal,-,False,False,0.000000,-1.000000,-0.000000,4.000000,-0.000000,0.000000,-0.000000
4,68.000000,14.000000,6.000000,0.000000,-10.000000,0.000000,-27.000000,0.000000,False,True,Superoptimal,-,False,False,0.000000,-1.000000,0.000000,4.000000,0.000000,0.000000,0.000000
5,68.000000,8.000000,0.000000,12.000000,2.000000,0.000000,-33.000000,0.000000,False,True,Superoptimal,-,False,False,0.000000,-1.000000,0.000000,4.000000,0.000000,0.000000,0.000000
6,68.000000,0.000000,-8.000000,28.000000,18.000000,0.000000,-41.000000,0.000000,False,True,Superoptimal,-,False,False,0.000000,-1.000000,0.000000,4.000000,0.000000,0.000000,0.000000
7,68.000000,9.000000,1.000000,10.000000,0.000000,0.000000,-32.000000,0.000000,False,True,Superoptimal,-,False,False,0.000000,-1.000000,0.000000,4.000000,-0.000000,-0.000000,-0.000000
8,60.000000,0.000000,0.000000,20.000000,10.000000,-8.000000,-25.000000,0.000000,False,False,-,-,False,False,0.000000,0.000000,0.000000,3.000000,-1.000000,1.000000,0.000000
9,50.000000,5.000000,15.000000,0.000000,-10.000000,-18.000000,0.000000,0.000000,False,True,Superoptimal,Suboptimal,True,False,0.000000,0.000000,0.666667,2.000000,0.000000,0.000000,0.333333
